<img src="https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Vectorización


In [27]:
import numpy as np

In [28]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * (np.linalg.norm(b)))

### Datos

In [29]:
corpus = np.array(['que dia es hoy', 'martes el dia de hoy es martes', 'martes muchas gracias'])
#corpus = np.array(['hola como estas', 'hola como estas y como llego al subte'])
#corpus = np.array(['que bueno que esta', 'esta muy bueno','esta excelente'])

Documento 1 --> que dia es hoy \
Documento 2 --> martes el dia de hoy es martes \
Documento 3 --> martes muchas gracias

### 1 - Obtener el vocabulario del corpus (los términos utilizados)
- Cada documento transformarlo en una lista de términos
- Armar un vector de términos no repetidos de todos los documentos

In [30]:
def crear_vocabulario(corpus):
    lista = []
    for documento in corpus:
        lista.extend(documento.split())
    print ("Lista de terminos:", lista)

    # Transformo la lista en Set para quedarme con sus elementos unicos.
    vocabulario = set(lista)
    vocabulario_lista = sorted(list(vocabulario))
    print("Vocabulario: ", vocabulario_lista, "\n")

    return vocabulario_lista

In [31]:
vocab = crear_vocabulario(corpus)

Lista de terminos: ['que', 'dia', 'es', 'hoy', 'martes', 'el', 'dia', 'de', 'hoy', 'es', 'martes', 'martes', 'muchas', 'gracias']
Vocabulario:  ['de', 'dia', 'el', 'es', 'gracias', 'hoy', 'martes', 'muchas', 'que'] 



### 2- OneHot encoding
Data una lista de textos, devolver una matriz con la representación oneHotEncoding de estos

In [32]:
def OHE(corpus):
    vocabulario = crear_vocabulario(corpus)
    matriz_one_hot = np.zeros((len(corpus), len(vocabulario)), dtype=np.int32)
    for row_index, doc in enumerate(corpus):
        for termino in doc.split():
            if termino in vocabulario:
                col_index = vocabulario.index(termino)  # Encuentra el índice de la palabra en el vocabulario
                matriz_one_hot[row_index, col_index] = 1

    return matriz_one_hot

In [33]:
OHE(corpus)

Lista de terminos: ['que', 'dia', 'es', 'hoy', 'martes', 'el', 'dia', 'de', 'hoy', 'es', 'martes', 'martes', 'muchas', 'gracias']
Vocabulario:  ['de', 'dia', 'el', 'es', 'gracias', 'hoy', 'martes', 'muchas', 'que'] 



array([[0, 1, 0, 1, 0, 1, 0, 0, 1],
       [1, 1, 1, 1, 0, 1, 1, 0, 0],
       [0, 0, 0, 0, 1, 0, 1, 1, 0]])

### 3- Vectores de frecuencia
Data una lista de textos, devolver una matriz con la representación de frecuencia de estos

In [34]:
def VDF (corpus):
    vocabulario = crear_vocabulario(corpus)
    vector_de_frecuencia = np.zeros((len(corpus), len(vocabulario)), dtype=np.int32)
    for row_index, doc in enumerate(corpus):
        for termino in doc.split():
            if termino in vocabulario:
                col_index = vocabulario.index(termino)  # Encuentra el índice de la palabra en el vocabulario
                vector_de_frecuencia[row_index, col_index] += 1

    return vector_de_frecuencia

In [35]:
VDF(corpus)

Lista de terminos: ['que', 'dia', 'es', 'hoy', 'martes', 'el', 'dia', 'de', 'hoy', 'es', 'martes', 'martes', 'muchas', 'gracias']
Vocabulario:  ['de', 'dia', 'el', 'es', 'gracias', 'hoy', 'martes', 'muchas', 'que'] 



array([[0, 1, 0, 1, 0, 1, 0, 0, 1],
       [1, 1, 1, 1, 0, 1, 2, 0, 0],
       [0, 0, 0, 0, 1, 0, 1, 1, 0]])

### 4- TF-IDF
Data una lista de textos, devolver una matriz con la representacion TFIDF

In [36]:
def tf_idf(corpus):
    TF = VDF(corpus)
    DF = np.sum(OHE(corpus), axis=0)
    IDF = np.round(np.log(len(corpus)/DF), 3)

    return(TF*IDF)

In [68]:
tf_idf(corpus)

Lista de terminos: ['que', 'dia', 'es', 'hoy', 'martes', 'el', 'dia', 'de', 'hoy', 'es', 'martes', 'martes', 'muchas', 'gracias']
Vocabulario:  ['de', 'dia', 'el', 'es', 'gracias', 'hoy', 'martes', 'muchas', 'que'] 

Lista de terminos: ['que', 'dia', 'es', 'hoy', 'martes', 'el', 'dia', 'de', 'hoy', 'es', 'martes', 'martes', 'muchas', 'gracias']
Vocabulario:  ['de', 'dia', 'el', 'es', 'gracias', 'hoy', 'martes', 'muchas', 'que'] 



array([[0.   , 0.405, 0.   , 0.405, 0.   , 0.405, 0.   , 0.   , 1.099],
       [1.099, 0.405, 1.099, 0.405, 0.   , 0.405, 0.81 , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 1.099, 0.   , 0.405, 1.099, 0.   ]])

### 5 - Comparación de documentos
Realizar una funcion que reciba el corpus y el índice de un documento y devuelva los documentos ordenados por la similitud coseno

In [99]:
def comparar_documentos(corpus, indice):
    
    # Calcular el vector de Frecuencia (Se puede utilizar cualquier otro metodo)
    vdf = VDF(corpus)

    # Calcular la similitud de coseno
    similaridad = []
    for document in vdf:
        similaridad.append(cosine_similarity(vdf[indice], document))

    # Reordenar los documentos por similitud
    enumerated_list = list(enumerate(similaridad))
    sorted_list = sorted(enumerated_list, key=lambda x: x[1], reverse=True)
    indices_ordenados = [index for index, value in sorted_list]

    # Imprimir
    for idx in indices_ordenados:
        print (corpus[idx], similaridad[idx])

    return

In [102]:
comparar_documentos(corpus, 1)

Lista de terminos: ['que', 'dia', 'es', 'hoy', 'martes', 'el', 'dia', 'de', 'hoy', 'es', 'martes', 'martes', 'muchas', 'gracias']
Vocabulario:  ['de', 'dia', 'el', 'es', 'gracias', 'hoy', 'martes', 'muchas', 'que'] 

martes el dia de hoy es martes 1.0
que dia es hoy 0.5
martes muchas gracias 0.3849001794597505
